In [1]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  %matplotlib inline
else:
    print('Not running on CoLab')
    %matplotlib notebook
    import os
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Not running on CoLab


In [2]:
import time
from os import path

from matplotlib.animation import FFMpegWriter

from funtion_file import *
from ks_funtions import *
from tensorflow.keras.metrics import Precision, Recall

import keras as ks

In [3]:
row_len = 15
col_len = 20
kernel_size = (2, 2)
min_max_line_size = [(1,5),(1,7)]
rotate = True
num_of_mat = 500
fades_per_mat = 8
num_neuron = 256
new_background = True


matrix_lister = MatrixLister(row_len, col_len, kernel_size, min_max_line_size, rotate, num_of_mat, fades_per_mat, new_background)

In [4]:
checkpoint_filepath = 'weights.h5'
model_checkpoint_callback = ks.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True,
                                                         monitor='loss', mode='min', save_best_only=True)

model = build_model(row_len, col_len, 32, fades_per_mat)
optimizer = ks.optimizers.Adam()
model.compile(optimizer=optimizer, loss=custom_weighted_loss, metrics=[Precision(name='precision'), Recall(name='recall')])

In [5]:
if path.exists('weights_good.h5'):
    model.load_weights('weights_good.h5')
    print('Loaded weights')

Loaded weights


In [7]:
batch_size = 2**12
epochs = 100

generator = DataGenerator(matrix_lister, batch_size, fades_per_mat, num_of_mat)

start = time.time()
hist = model.fit(generator, epochs=epochs, callbacks=[model_checkpoint_callback])

print(time.time() - start)

Epoch 1/100
1/1 [==============================] - 1s 848ms/step - loss: 3.4630e-04 - precision: 0.9834 - recall: 1.0000
Epoch 2/100
1/1 [==============================] - 1s 755ms/step - loss: 3.8125e-04 - precision: 0.9829 - recall: 0.9998
Epoch 3/100
1/1 [==============================] - 1s 807ms/step - loss: 3.7560e-04 - precision: 0.9850 - recall: 0.9998
Epoch 4/100
1/1 [==============================] - 1s 805ms/step - loss: 3.4154e-04 - precision: 0.9826 - recall: 1.0000
Epoch 5/100
1/1 [==============================] - 1s 779ms/step - loss: 3.6817e-04 - precision: 0.9841 - recall: 0.9997
Epoch 6/100
1/1 [==============================] - 1s 779ms/step - loss: 4.4640e-04 - precision: 0.9819 - recall: 0.9996
Epoch 7/100
1/1 [==============================] - 1s 819ms/step - loss: 4.7047e-04 - precision: 0.9847 - recall: 0.9992
Epoch 8/100
1/1 [==============================] - 1s 946ms/step - loss: 3.1858e-04 - precision: 0.9845 - recall: 1.0000
Epoch 9/100
1/1 [===============

KeyboardInterrupt: 

In [22]:
model.save_weights('weights_good.h5')

In [8]:
ani = matrix_lister.plot_matrices(model, 3, interval=200)
plt.show()

1/1 [==============================] - 0s 268ms/step


<IPython.core.display.Javascript object>

In [11]:
if 'google.colab' in str(get_ipython()):
    f = r"animation.mp4"
    writervideo = FFMpegWriter(fps=24)
    ani.save(f, writer=writervideo)

In [9]:
plot_training_history(hist, with_val=False)

<IPython.core.display.Javascript object>